# geoDB access

The geoDB is a service provided by the [Euro Data Cube project](https://eurodatacube.com) (EDC) as a paid service. It comes
with a Python client that provides high level access to your data and a certain amount of space in a PostGreSQL database. For exploring data you will need a geoDB subscription which you can purchase at the [EDC market place](https://eurodatacube.com/marketplace).

You can access the service in two ways:

- By using the Jupyter Python notebook provided by EDC Marketplace (configuration-free, ```geodb = GeoDBClient()```)
- By using your own Jupyter notebook or Python script by providing a client id and secret to the GeoDBClient (```geodb = GeoDBClient(client_id="myid", client_secret="mysecet")```)

The client ID and secret are provided by EDC. You will find them in your EDC Marketplace account section. You can also provide the credentials
via system environment variables (GEODB_AUTH_CLIENT_ID and GEODB_AUTH_CLIENT_SECRET). These variables can be supplied via a .env file.

## geoDB indexes

You may create indexes on properties of your collections in geoDB. These indexes potentially speed up queries drastically, but adding too many indexes on a collection might also hamper its performance. Please use with care, and use only if you know what you are doing. This notebook will demonstrate how to work with indexes.

Note that this notebook does not show any performance tests; this is due to the fact that in order for indexes to work efficiently, the collection sizes must be significantly higher than of our demonstration collections.

In [ ]:
from xcube_geodb.core.geodb import GeoDBClient

##  Login from your machine

Install xcube geoDB with the command:

    `conda install xcube_geodb -c conda-forge`

In [ ]:
# uncomment the following lines and fill in credentials
#client_id=YourID
#client_secret=YourSecret
#geodb = GeoDBClient(client_id=client_id, client_secret=client_secret, auth_mode="client-credentials", server_url='https://xcube-geodb.stage.brockmann-consult.de', auth_aud='https://xcube-users.brockmann-consult.de/api/v2')

## Login in EDC environment

In [ ]:
### comment if not on EDC

geodb = GeoDBClient()

Show current user, and create a collection, as shown in the notebook `Manage collections`:

In [ ]:
print(geodb.whoami)
collection_name = "land_use"
collections = {
    collection_name: {"crs": 3794, "properties": {"RABA_PID": "float",
                                                  "RABA_ID": "float",
                                                  "D_OD": "date"
                                                  }
                      }
}
geodb.create_collections(collections, clear=True)
import geopandas
gdf = geopandas.read_file('data/sample/land_use.shp')
m = geodb.insert_into_collection(collection_name, gdf)
str(m)

Create an index on the geometry of the new collection:

In [ ]:
m = geodb.create_index(collection_name, 'geometry')
str(m)

Show the indexes on the collection. Note that there are two indexes: geoDB automatically creates an index on the collection's id.

In [ ]:
print(geodb.show_indexes(collection_name))

Running a sample query which does a geometric lookup; for collections of the size of the demonstration collection, the query will not use the index, but if you run a similar query on a larger collection, you will experience huge performance gains.

In [ ]:
geodb.get_collection_by_bbox(collection_name, bbox=(453952, 91124, 456136, 93054), bbox_crs=3794)

Remove the index again. This is useful e.g. if you have created too many indexes, and thus ingestion speed is reduced.

In [ ]:
geodb.remove_index(collection_name, 'geometry')

Finally, clean up:

In [ ]:
geodb.drop_collection(collection_name)